# Crime Database

In this project we will build a database for storing data related with crimes that occurred in Boston. This dataset is provided by [Dataquest.io](https://www.dataquest.io/) and available in the file "boston.csv"

The goal of this project is to create a database named **crime_db** with a table "boston_crimes" with appropriate datatypes for storing the data from the "boston.csv" file. We'll be creating the table inside a schema named **crimes**. We'll also create the *readonly* and *readwrite* groups with the appropriate privileges. Finally, we'll create some users for each of these groups.

Database final state will look like this:
![final_state](db_diag.png)

## Preparations

Because we don't have **crime_db** for now it's time to create it!
There is several possible ways to create postgreSQL database:
- with *psycopg2* Python package (connect to existing db and create new one).
- using Linux Bash/Windows PowerShell/iOS CLI (connect to psql shell and create db).

PErsonally I'll use Linux approach (currntly working on Ubuntu 20) and with the help of Bash we'll create new database.
I'll leave basic "postgres" user for now.

![Picture of creating DB via bash](bash_crime_db.png)

If you firmly sure to create database with the Python package, then your code will look like this for base Postges database:

```
import psycopg2

conn = psycopg2.connect(dbname="postgres",
                        user="postgres",
                        host="localhost",
                        password="postgres")
conn.autocommit = True
cur = conn.cursor()
cur.execute("CREATE DATABASE crime_db;")
conn.close()
```
<br>
<br>
Now let's just connect to previously created database and make new schema:

In [5]:
import psycopg2
from csv import reader

In [3]:
# Password was created earlier in psql shell
# with ALTER USER postgres PASSWORD 'postgres'; query.

conn = psycopg2.connect(dbname="crime_db",
                        user="postgres",
                        host="localhost",
                        password="postgres")
cur = conn.cursor()
cur.execute("CREATE SCHEMA crimes;")
conn.commit()

There's no need to close this connection now. We'll work with it until project goals will be reached.

## File exploration

Before we created any tables let's see what data lies inside "boston.csv".<br>
This step will allow us to use proper data types in our future **crime_db** tables.

In [40]:
with open("boston.csv") as file:
    r_file = list(reader(file))
    header = r_file[0]
    first_three = r_file[1:4]

header

['incident_number',
 'offense_code',
 'description',
 'date',
 'day_of_the_week',
 'lat',
 'long']

Our database has columns:
* **incident_number** - identifier of the crime.
* **offense_code** - numeric identifier code for the committed crime.
* **description** - description of the crime.
* **date** - date on which the crime happened.
* **day_of_the_week** - day of the week corresponding to **date**.
* **lat** - latitude coordinates of crime location.
* **long** - latitude coordinates of crime location.

In [41]:
first_three

[['1',
  '619',
  'LARCENY ALL OTHERS',
  '2018-09-02',
  'Sunday',
  '42.35779134',
  '-71.13937053'],
 ['2',
  '1402',
  'VANDALISM',
  '2018-08-21',
  'Tuesday',
  '42.30682138',
  '-71.06030035'],
 ['3',
  '3410',
  'TOWED MOTOR VEHICLE',
  '2018-09-03',
  'Monday',
  '42.34658879',
  '-71.07242943']]

After looking on first three rows we can name values data types (in order):
- integer number
- integer number
- string
- date
- string
- decimal number
- decimal number

We'll now find amount of unique elements in columns to see if we can turn categorical data to enumerated datatypes.

In [19]:
# Helper function that makes sets:
def count_set_from_columns(file, col_id):
    col_values = []
    for row in file[1:]:
        col_values.append(row[col_id])
    return set(col_values)

In [39]:
with open("boston.csv") as file:
    r_file = list(reader(file))

# We already have "header" value from previous boston.csv reading:
for ind in range(len(header)):
    result = count_set_from_columns(r_file, ind)
    print(f"Column '{header[ind]}':", len(result))

Column 'incident_number': 298329
Column 'offense_code': 219
Column 'description': 239
Column 'date': 1177
Column 'day_of_the_week': 7
Column 'lat': 18177
Column 'long': 18177


Looks like the only column for numerical transformation is "day_of_the_week".

Also let's count max length of the longest value in 'description' column to se how we can specify data type maximum length.

In [38]:
max_len_desc = 0
cr_description = count_set_from_columns(r_file, 2)
for l in cr_description:
    max_len_desc = max(max_len_desc, len(l))

print(max_len_desc)

58


## DB table creation

Now we have all information needed to create db table, so let's do it.

In [42]:
# New data type for days of the week numerical conversion:
cur.execute("""
CREATE TYPE weekdays_enum
AS ENUM ('Monday', 'Tuesday','Wednesday', 'Thursday',
         'Friday', 'Saturday', 'Sunday');
""")

# Only 'description' will get max length:
cur.execute("""
CREATE TABLE crimes.boston_crimes (
    incident_number INTEGER PRIMARY KEY,
    offense_code INTEGER,
    description VARCHAR(100),
    date DATE,
    day_of_the_week weekdays_enum,
    lat DECIMAL,
    lon DECIMAL);
""")

conn.commit()

## Data loading to DB

Let's load data from "boston.csv" to our newly created table.

In [49]:
with open("boston.csv") as f:
    cur.copy_expert("COPY crimes.boston_crimes FROM STDIN WITH CSV HEADER DELIMITER as ',';", f)
    conn.commit()

And check for successfully uploaded data:

In [57]:
cur.execute("SELECT * FROM crimes.boston_crimes LIMIT 3;")
cur.fetchall()

[(1,
  619,
  'LARCENY ALL OTHERS',
  datetime.date(2018, 9, 2),
  'Sunday',
  Decimal('42.35779134'),
  Decimal('-71.13937053')),
 (2,
  1402,
  'VANDALISM',
  datetime.date(2018, 8, 21),
  'Tuesday',
  Decimal('42.30682138'),
  Decimal('-71.06030035')),
 (3,
  3410,
  'TOWED MOTOR VEHICLE',
  datetime.date(2018, 9, 3),
  'Monday',
  Decimal('42.34658879'),
  Decimal('-71.07242943'))]

Let's also check for our "day_of_the_week" column type is correct as we created above.

In [58]:
print(cur.description)

(Column(name='incident_number', type_code=23), Column(name='offense_code', type_code=23), Column(name='description', type_code=1043), Column(name='date', type_code=1082), Column(name='day_of_the_week', type_code=16396), Column(name='lat', type_code=1700), Column(name='lon', type_code=1700))


In [53]:
cur.execute("SELECT typname FROM pg_catalog.pg_type WHERE oid = 16396;")
cur.fetchone()[0]

'weekdays_enum'

All seems to be good!

## Revoking Public Privileges and Groups Creation

Now it is time to handle users. Our goal is to create the two user groups: *readonly* and *readwrite*.<br>
By following the least privilege principle, the first step in doing so is to make sure that there are no privileges inherited from the public group and on the public schema.

In [66]:
# Don't forget to use conn.rollback() if something wrong with the query.

cur.execute("REVOKE ALL ON SCHEMA crimes FROM public;")
cur.execute("REVOKE ALL ON DATABASE crime_db FROM public;")

Both groups need CONNECT to database privilege - otherwise, they won't be able to do anything.
Same thing with USAGE on schema privilege.

*Readonly* group will get only SELECT option.

In [67]:
cur.execute("CREATE GROUP readonly NOLOGIN;")
cur.execute("GRANT CONNECT ON DATABASE crime_db TO readonly;")
cur.execute("GRANT USAGE ON SCHEMA crimes TO readonly;")
cur.execute("GRANT SELECT ON ALL TABLES IN SCHEMA crimes TO readonly;")
conn.commit()

*Readwrite* group will get SELECT, INSERT, DELETE and UPDATE options. We'll not grant DROP privilege to this groгp, because it's more related to the database engineer position. 

In [68]:
cur.execute("CREATE GROUP readwrite NOLOGIN;")
cur.execute("GRANT CONNECT ON DATABASE crime_db TO readwrite;")
cur.execute("GRANT USAGE ON SCHEMA crimes TO readwrite;")
cur.execute("GRANT SELECT ON ALL TABLES IN SCHEMA crimes TO readwrite;")
conn.commit()

## Add Users

Our final step is to add several users of our database.

In [69]:
cur.execute("CREATE USER data_analyst WITH PASSWORD 'dickbutt';")
cur.execute("GRANT readonly TO data_analyst;")

cur.execute("CREATE USER data_scientist WITH PASSWORD 'suchdoge';")
cur.execute("GRANT readwrite TO data_scientist;")

conn.commit()

Let's check privileges that we granted to groups.<br>

In the pg_roles table we will check some database related privileges and for that we will look at the following columns:

    rolname: The name of the user / group that the privilege refers to.
    rolsuper: Whether this user / group is a super user. It should be set to False on every user / group that we have created.
    rolcreaterole: Whether user / group can create users, groups or roles. It should be False on every user / group that we have created.
    rolcreatedb: Whether user / group can create databases. It should be False on every user / group that we have created.
    rolcanlogin: Whether user / group can login. It should be True on the users and False on the groups that we have created.

In the information_schema.table_privileges we will check privileges related to SQL queries on tables. We will list the privileges of each group that we have created.

In [109]:
cur.execute("""
    SELECT rolname, rolsuper, rolcreaterole, rolcreatedb, rolcanlogin FROM pg_roles
    WHERE rolname IN ('readonly', 'readwrite', 'data_analyst', 'data_scientist');
""")
for user in cur:
    print(user)

('readonly', False, False, False, False)
('readwrite', False, False, False, False)
('data_analyst', False, False, False, True)
('data_scientist', False, False, False, True)


In [111]:
cur.execute("""
    SELECT grantee, privilege_type
    FROM information_schema.table_privileges
    WHERE grantee IN ('readonly', 'readwrite');
""")
for user in cur:
    print(user)

('readwrite', 'INSERT')
('readwrite', 'SELECT')
('readwrite', 'UPDATE')
('readwrite', 'DELETE')


It seems like there must be more privileges. That's happened due to the copy/paste without proper attention.<br>
Let's fix it!

In [73]:
cur.execute("GRANT SELECT, INSERT, DELETE, UPDATE ON ALL TABLES IN SCHEMA crimes TO readwrite;")
conn.commit()

In [74]:
cur.execute("""
SELECT grantee, privilege_type
FROM information_schema.table_privileges
WHERE grantee = 'readwrite';""")
cur.fetchall()

[('readwrite', 'INSERT'),
 ('readwrite', 'SELECT'),
 ('readwrite', 'UPDATE'),
 ('readwrite', 'DELETE')]

Access rights are looking good now.

conn.close()

## Let's make some queries!

All steps are left behind and it's time to check connection.<br>
<br>
Let's connect as data_scientist and make some queries.

In [76]:
conn_test = psycopg2.connect(dbname="crime_db",
                             user="data_scientist",
                             host="localhost",
                             password="suchdoge")
cur = conn_test.cursor()

What is the time gap of our data?

In [87]:
cur.execute("SELECT MIN(date), MAX(date) FROM crimes.boston_crimes;")
cur.fetchall()

[(datetime.date(2015, 6, 15), datetime.date(2018, 9, 3))]

Are there some records with kills?

In [102]:
cur.execute("""
SELECT *  
FROM crimes.boston_crimes
WHERE description LIKE '%KILL%';
""")
cur.fetchall()

[(237398,
  112,
  'KILLING OF FELON BY POLICE',
  datetime.date(2016, 2, 12),
  'Friday',
  Decimal('42.33367922'),
  Decimal('-71.09187755'))]

Find max record number to add one more record:

In [95]:
cur.execute("SELECT MAX(incident_number) FROM crimes.boston_crimes;")
cur.fetchall()

[(298329,)]

In [100]:
cur.execute("""
INSERT INTO crimes.boston_crimes VALUES (
        298330,
        0101,
        'OVERDOSE WITH MEMES',
        '2020, 8, 31',
        'Friday',
        59.859183,
        30.188727
);
""")
conn_test.commit()

And check that record was made properly:

In [103]:
cur.execute("""
SELECT *  
FROM crimes.boston_crimes
WHERE offense_code = 0101;
""")
cur.fetchall()

[(298330,
  101,
  'OVERDOSE WITH MEMES',
  datetime.date(2020, 8, 31),
  'Friday',
  Decimal('59.859183'),
  Decimal('30.188727'))]

It was not good idea to pass an offence code "0101" as during record to database Postgres has turned it to "101".
But still query found our record.

In [114]:
conn_test.close()